In [1]:
len([0,2,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21, 22, 23, 25, 26, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46])

34

In [5]:
import sys
!{sys.executable} -m pip install selenium

  Using cached selenium-3.141.0-py2.py3-none-any.whl (904 kB)


In [85]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys 
import time

In [38]:
url = "https://polka.d3.ru/"

In [39]:
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--headless')
driver = webdriver.Chrome("chromedriver/chromedriver.exe", options=options)

In [50]:
driver.get(url)

scrolls = 0
max_scrolls = 20

SCROLL_PAUSE_TIME = 0.5

# Get scroll height
last_height = driver.execute_script("return document.body.scrollHeight")

while scrolls < max_scrolls:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    scrolls += 1
    if new_height == last_height:
        break
    last_height = new_height

#driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
page_source = driver.page_source

In [51]:
soup = BeautifulSoup(page_source, 'html.parser')

In [62]:
post_dict = {
    "title" : [],   #title of the post
#         "score" : [],   # score of the post
    "id" : [],      # unique id of the post
#         "url" : [],     #url of the post
#         "comms_num": [],   #the number of comments on the post
#         "created" : [],  #timestamp of the post
    "body" : []         # the descriptionof post
}

comments_dict = {
    "comment_id" : [],      #unique comm id
    "comment_parent_id" : [],   # comment parent id
    "comment_body" : [],   # text in comment
    "comment_link_id" : []  #link to the comment
}

links = []
i = 0  
for post in soup.findAll('span', attrs= {'class':"b-post-title__container"}):
    for a in post.findAll('a', href=True):
        if i == 0:
            continue
        post_dict['title'].append(a.text)
        links.append(a['href'])
        post_dict['id'].append(a['href'][-8:-1])
    i += 1

In [244]:
j = 0
for link in links:
    print(j)
    j += 1
    driver.get(link)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'lxml')
    for post in soup.findAll('div', attrs= {'class':"b-post__content-cell x-ugc"}):
        post_dict['body'].append(post.text)
    i = 0
    for parent in driver.find_element_by_id('b-comment-root').find_elements_by_xpath('./div'):
        root = parent.get_attribute('id')[-8:]
        comments = parent.find_elements_by_xpath('./div')
        comments_dict["comment_id"].append(root)
        comments_dict["comment_parent_id"].append(link[-8:-1])
        comments_dict['comment_body'].append(comments.pop(0).text)
        comments_dict['comment_link_id'].append(link[-8:-1])
        for comment in comments:
            comments_dict["comment_id"].append(i) ## see if i can get this?
            comments_dict["comment_parent_id"].append(root) ## only can see the FULL parent. 
            comments_dict['comment_body'].append(comment.text)
            comments_dict['comment_link_id'].append(link[-8:-1])
            i += 1

0
1
2


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="b-comment-root"]"}
  (Session info: headless chrome=88.0.4324.146)


In [229]:
link

'https://polka.d3.ru/i-togda-u-vas-ostaiutsia-dva-varianta-2110128/'

In [231]:
## Troubleshot
link = links[0]
driver.get(link)
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
page_source = driver.page_source
soup = BeautifulSoup(page_source, 'html.parser')
for post in soup.findAll('div', attrs= {'class':"b-post__content-cell x-ugc"}):
    post_dict['body'].append(post.text)

In [149]:
for comment in driver.find_elements_by_xpath('//div[@class="b-comment__content-cell x-ugc"]'):
    comments_dict['comment_link_id'].append(link[-8:-1])
    comments_dict['comment_body'].append(comment.text)

In [232]:
i = 0
for parent in driver.find_element_by_id('b-comment-root').find_elements_by_xpath('./div'):
    root = parent.get_attribute('id')[-8:]
    comments = parent.find_elements_by_xpath('./div')
    comments_dict["comment_id"].append(root)
    comments_dict["comment_parent_id"].append(link[-8:-1])
    comments_dict['comment_body'].append(comments.pop(0).text)
    comments_dict['comment_link_id'].append(link[-8:-1])
    for comment in comments:
        comments_dict["comment_id"].append(i) ## see if i can get this?
        comments_dict["comment_parent_id"].append(root) ## only can see the FULL parent. 
        comments_dict['comment_body'].append(comment.text)
        comments_dict['comment_link_id'].append(link[-8:-1])
        i += 1

In [234]:
len(driver.find_element_by_id('b-comment-root').find_elements_by_xpath('./div'))

43

In [242]:
for parent in driver.find_element_by_id('b-comment-root').find_elements_by_xpath('./div'):
    root = parent.get_attribute('id')[-8:]
    print(root)
    print()
    comments = parent.find_elements_by_xpath('./div')
    print(len(comments))
    print(comments[1].get_attribute('id')[-8:])
    sub_comments = comments[0].find_elements_by_xpath('./div')
    print(len(sub_comments))
    print(sub_comments[1].get_attribute('id')[-8:])
    sub_sub_comments = sub_comments[2].find_elements_by_xpath('./div')
    print(len(sub_sub_comments))
    
#     print()
#     for comment in comments:
#         print(comment.text)
    break

28736104

9
28736835
3

2


In [223]:
comments_dict['comment_body']

['Идеальная ситуация для России — парламентская республика. Вообще безо всяких Навальных во главе. Народец–то здесь, как оказывается, большой любитель делать себе царя из любого понравившегося индивида — делегировать одному избранному все полномочия, чтобы тот в одиночку сделал для них заебись, а сами они при этом продолжали сидеть по своим печкам ровно, палец о палец при этом не ударив, со всеми вытекающими. Иначе будет та же цикличность — плохой царь, хороший царь и так во веки веков. Аминь.',
 'Kyojima, Навальный по моему не про царя, а про свободные выборы. Безусловно, у него огромные шансы, но это не аксиома.',
 'Kyojima, Да, парламентская республика. К этому пришли и Британская и Германская, да что говорить и Американская империи. Но ее создание связано с огромным объемом реформ, который фактически означает принятие новой Конституции и коренное изменение уклада. Просто, через запятую:\n— коренная переделка институтов, независимый суд, сокращение ФСБ (фактически люстрации), отмена

In [ ]:
x